1. Explonatory Data Analysis and Understanding the Dataset

In [1]:
#importing necessary libraries
import pandas as pd 
import numpy as np
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql.functions import length, max
from transformers import BertTokenizer, BertModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler,Word2Vec,StringIndexer, Tokenizer,HashingTF,IDF,RegexTokenizer,CountVectorizer,StopWordsRemover
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import regexp_replace, col, count, when, size
from pyspark.sql.functions import split
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier

2025-03-23 22:41:10.028449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742749870.076873   52449 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742749870.092385   52449 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-23 22:41:10.202991: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .getOrCreate()


your 131072x1 screen size is bogus. expect trouble
25/03/23 22:41:19 WARN Utils: Your hostname, nipuna resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/23 22:41:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/23 22:41:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
#import dataset to a dataframe in spark
dataset=spark.read.csv('Mendeley dataset.csv',header=True,inferSchema=True)
dataset=dataset.select('lyrics','genre')

In [4]:
dataset = dataset.withColumn('lyrics', regexp_replace(col('lyrics'), '[^a-zA-Z\\s]', ''))
dataset = dataset.withColumn("lyrics", regexp_replace(col("lyrics"), r"\d+", ''))

In [5]:
#dataframe visualization
dataset.show(10)

+--------------------+-----+
|              lyrics|genre|
+--------------------+-----+
|hold time feel br...|  pop|
|believe drop rain...|  pop|
|sweetheart send l...|  pop|
|kiss lips want st...|  pop|
|till darling till...|  pop|
|convoy light dead...|  pop|
|piece mindin worl...|  pop|
|care moment hold ...|  pop|
|lonely night surr...|  pop|
|tear heart seat s...|  pop|
+--------------------+-----+
only showing top 10 rows



In [17]:
#column names and their properties
dataset.printSchema()

root
 |-- lyrics: string (nullable = true)
 |-- genre: string (nullable = true)



In [18]:
#check the maximum length of a lyrics
max_length = dataset.select(max(length(dataset["lyrics"]))).collect()[0][0]
print("Max lyric length:", max_length)

Max lyric length: 1714


2. Word2Vec Encoding for Encoding the Lyrics

In [ ]:
# dataset = dataset.withColumn("lyrics_tokens", split(dataset["lyrics"], " "))
# indexer = StringIndexer(inputCol="genre", outputCol="label")
# word2Vec = Word2Vec(vectorSize=300, minCount=1, inputCol="lyrics_tokens", outputCol="lyrics_embedding")
# lr = LogisticRegression(featuresCol="lyrics_embedding", labelCol="label")
# pipeline = Pipeline(stages=[indexer, word2Vec, lr])
# train_data, test_data = dataset.randomSplit([0.8, 0.2], seed=42)


In [6]:
indexer = StringIndexer(inputCol="genre", outputCol="label")
tokenizer = Tokenizer(inputCol="lyrics", outputCol="tokens")
stop_words_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
word2Vec = Word2Vec(vectorSize=300, minCount=1, inputCol="filtered_tokens", outputCol="lyrics_embedding")
lr = LogisticRegression(featuresCol="lyrics_embedding", labelCol="label")
pipeline = Pipeline(stages=[indexer,tokenizer,stop_words_remover, word2Vec, lr])
train_data, test_data = dataset.randomSplit([0.8, 0.2], seed=42)

In [7]:
model = pipeline.fit(train_data)

25/03/23 22:41:51 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [13]:
# Make predictions on test_data
predictions = model.transform(test_data)
# Create the evaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy"
)

# Evaluate accuracy on the test set
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.35844296269598125


The model accuracy of logistic regression is low. There are two approaches to increase accuracy. Change the encoding method because Word2Vex is static encoding. We can use advanced trained model. Another option is to use advanced model for classifications.

3. Random forest Model 

In [ ]:
dataset=spark.read.csv('Mendeley dataset.csv',header=True,inferSchema=True)
dataset=dataset.select('lyrics','genre')
dataset = dataset.withColumn('lyrics', regexp_replace(col('lyrics'), '[^a-zA-Z\\s]', ''))
dataset = dataset.withColumn("lyrics", regexp_replace(col("lyrics"), r"\d+", ''))
train_data, test_data = dataset.randomSplit([0.8, 0.2], seed=42)

In [12]:
# Define pipeline stages
tokenizer = Tokenizer(inputCol="lyrics", outputCol="tokens")
stop_words_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
cv = CountVectorizer(inputCol="filtered_tokens", outputCol="raw_features", vocabSize=250, minDF=2)
idf = IDF(inputCol="raw_features", outputCol="tfidf_vector")
indexer = StringIndexer(inputCol="genre", outputCol="label")
rf = RandomForestClassifier(featuresCol="tfidf_vector", labelCol="label")
pipeline = Pipeline(stages=[tokenizer, stop_words_remover, cv, idf, indexer, rf])


In [13]:
# 1️⃣1️⃣ Train Model
model = pipeline.fit(train_data)

In [14]:
# 1️⃣2️⃣ Make Predictions
predictions = model.transform(test_data)

In [15]:
# 1️⃣3️⃣ Evaluate Model
evaluator = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Random Forest Model Accuracy: {accuracy:.2f}")

Random Forest Model Accuracy: 0.27
